### Use pyserini to retrieve relevant data

In [1]:
from datasets import load_from_disk
import json
import os

/home/olagh48652/irg_course_assig/irg-programming/irg-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
corpus = load_from_disk("LitSearch_corpus_clean")

In [ ]:
## save corpus in a way that works with pyserini
output_dir = "corpus_jsonl"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "corpus.jsonl")

with open(output_path, "w", encoding="utf-8") as f:
    for item in corpus["full"]:
        doc_id = str(item["corpusid"])
        contents = (item["title"] or "") + " " + (item["abstract"] or "")
        f.write(json.dumps({"id": doc_id, "contents": contents}) + "\n")

print(" Saved corpus.jsonl for Pyserini indexing.")

 Saved corpus.jsonl for Pyserini indexing.


In [7]:
## get index using commandline 
import subprocess

index_dir = "pyserini_index"
os.makedirs(index_dir, exist_ok=True)
index_path = os.path.abspath(index_dir)

cmd = [
    "python", "-m", "pyserini.index.lucene",
    "--collection", "JsonCollection",
    "--input", "/home/olagh48652/irg_course_assig/irg_project/corpus_jsonl",
    "--index", index_path,
    "--generator", "DefaultLuceneDocumentGenerator",
    "--threads", "1",
    "--storePositions", "--storeDocvectors", "--storeRaw"
]

# Run the command
result = subprocess.run(cmd, capture_output=True, text=True)

# Print stdout and stderr
print(result.stdout)
print(result.stderr)

2025-10-15 13:57:00,868 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:205) - Setting log level to INFO
2025-10-15 13:57:00,876 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - ============ Loading Index Configuration ============
2025-10-15 13:57:00,877 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) - AbstractIndexer settings:
2025-10-15 13:57:00,878 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + DocumentCollection path: /home/olagh48652/irg_course_assig/irg_project/corpus_jsonl
2025-10-15 13:57:00,879 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + CollectionClass: JsonCollection
2025-10-15 13:57:00,880 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Index path: /home/olagh48652/irg_course_assig/irg_project/pyserini_index
2025-10-15 13:57:00,881 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Threads: 1
2025-10-15 13:57:00,882 INFO  [main] index.AbstractIndexer (Abstra

In [3]:
dataset_query = load_from_disk("LitSearch_query")

query_0 = dataset_query["full"][0]["query"]
print("query_0: ", query_0)

query_1 = dataset_query["full"][1]["query"]
print("query_1: ", query_1)

query_0:  Are there any research papers on methods to compress large-scale language models using task-agnostic knowledge distillation techniques?
query_1:  Are there any resources available for translating Tunisian Arabic dialect that contain both manually translated comments by native speakers and additional data augmented through methods like segmentation at stop words level?


In [1]:
## get run file
from pyserini.search.lucene import LuceneSearcher
import json

index_path = "/home/olagh48652/irg_course_assig/irg_final_project/pyserini_index"
# Initialize BM25 searcher
lucene_bm25_searcher = LuceneSearcher(index_path)
lucene_bm25_searcher.set_bm25(k1=0.9, b=0.4)  # Optional tuning

# Example: list of query dicts (replace with your real requests)
requests = [
    {"request_id": 1, "query": query_0},
    {"request_id": 2, "query": query_1}
]

# File to save top-3 results
output_file = "litsearch_topk_results.run"
k = 50
method_name = "bm25"
with open(output_file, "w", encoding="utf-8") as out_f:
    for req in requests:
        query_text = req["query"]
        request_id = req["request_id"]
        
        hits = lucene_bm25_searcher.search(query_text, k=k)

        # topk = [
        #     {"doc_id": hit.docid, "score": hit.score, "rank": rank + 1}
        #     for rank, hit in enumerate(hits)
        # ]
        
        for rank, hit in enumerate(hits, start=1):
            doc_id = hit.docid
            rank = rank
            score = hit.score    
            line = f"{request_id} Q0 {doc_id} {rank} {score:.6f} {method_name}\n"
            out_f.write(line)
        


print(f" Top-k results per query written to {output_file}")


C:\Users\keith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\keith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x


AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\keith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\keith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1075, in launch

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\keith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\keith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1075, in launch

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
# save in json
from pyserini.search.lucene import LuceneSearcher
import json

# Initialize BM25 searcher
lucene_bm25_searcher = LuceneSearcher(index_path)
lucene_bm25_searcher.set_bm25(k1=0.9, b=0.4)  # Optional tuning

# Example: list of query dicts (replace with your real requests)
requests = [
    {"request_id": 1, "query": query_0},
    {"request_id": 2, "query": query_1}
]

# File to save top-3 results
output_file = "litsearch_top3_results.jsonl"

with open(output_file, "w", encoding="utf-8") as out_f:
    for req in requests:
        query_text = req["query"]
        hits = lucene_bm25_searcher.search(query_text, k=3)

        top3 = [
            {"doc_id": hit.docid, "score": hit.score, "rank": rank + 1}
            for rank, hit in enumerate(hits)
        ]

        out_f.write(json.dumps({
            "qid": req["request_id"],
            "query": query_text,
            "top3": top3
        }) + "\n")

print(f"✅ Top-3 results per query written to {output_file}")